In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd


train = pd.read_json('D:/kaggle_data/dont-call-me-turkey/train.json')
test = pd.read_json('D:/kaggle_data/dont-call-me-turkey/test.json')
sample_submission = pd.read_csv('D:/kaggle_data/dont-call-me-turkey/sample_submission.csv')


train.info()
test.info()
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1195 entries, 0 to 1194
Data columns (total 5 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   audio_embedding                  1195 non-null   object
 1   is_turkey                        1195 non-null   int64 
 2   vid_id                           1195 non-null   object
 3   end_time_seconds_youtube_clip    1195 non-null   int64 
 4   start_time_seconds_youtube_clip  1195 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 46.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1196 entries, 0 to 1195
Data columns (total 4 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   audio_embedding                  1196 non-null   object
 1   vid_id                           1196 non-null   object
 2   end_time_seconds_youtube_clip    1196 non-null   int64 
 3   

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12733767209196716480
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17301057137865033811
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4928812339
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14146925948131669070
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9552504082486023639
physical_device_desc: "device: XLA_GPU device"
]


In [3]:
print(len(train['audio_embedding'][0]))

10


In [4]:
def length(x):
    return len(x['audio_embedding'])

In [5]:
def length2(x):
    sum=0
    for i in range(0, len(x['audio_embedding'])):
        sum=sum+len(x['audio_embedding'][i])
    return sum/len(x['audio_embedding'])

In [6]:
train['audio_len2']=train.apply(length2,axis=1)

In [7]:
train['audio_len']=train.apply(length,axis=1)

In [8]:
#audio의 길이가 대부분 10
train.describe()

,is_turkey,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip,audio_len2,audio_len
count,1195.000000,1195.000000,1195.000000,1195.0,1195.000000
mean,0.410879,87.013389,77.085356,128.0,9.870293
std,0.492199,107.750860,107.697523,0.0,0.730351
min,0.000000,2.000000,0.000000,128.0,2.000000
25%,0.000000,40.000000,30.000000,128.0,10.000000
50%,0.000000,40.000000,30.000000,128.0,10.000000
75%,1.000000,90.000000,80.000000,128.0,10.000000
max,1.000000,600.000000,590.000000,128.0,10.000000


In [9]:
from keras.preprocessing.sequence import pad_sequences

In [10]:
x_val =train['audio_embedding']
y_val = train['is_turkey']
x_train =train['audio_embedding']
y_train = train['is_turkey']

In [11]:
x_train = pad_sequences(x_train, maxlen=10)
x_val = pad_sequences(x_val, maxlen=10)


test_data = test['audio_embedding'].tolist()
x_test = pad_sequences(test_data, maxlen=10)

In [12]:
from keras.models import Sequential
from tensorflow import keras
from keras.layers import BatchNormalization,Bidirectional,LSTM,Activation, Dense ,GlobalMaxPool1D, Dropout

In [13]:
model = Sequential()
model.add(BatchNormalization(input_shape=(10, 128)))
#양방향 LSTM
model.add(Bidirectional(LSTM(128, activation='relu',recurrent_dropout=0.1,return_sequences=True)))
model.add(Bidirectional(LSTM(128, activation='relu',recurrent_dropout=0.1,return_sequences=True)))
model.add(Bidirectional(LSTM(128, activation='relu',recurrent_dropout=0.1,return_sequences=False)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
history = model.fit(x_train, y_train,batch_size=300, epochs=70, validation_data=(x_val, y_val))


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 10, 128)           512       
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 256)           263168    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 10, 256)           394240    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               394240    
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 1,052,417
Trainable params: 1,052,161
Non-trainable params: 256
_________________________________________________________________
Epoch 1/70
4/4 [==============================] - 2s 473ms/step - loss: 0.5839 - accuracy: 0.7841 - val_loss: 

Epoch 41/70
4/4 [==============================] - 1s 203ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.2964 - val_accuracy: 0.9757
Epoch 42/70
4/4 [==============================] - 1s 204ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.4544 - val_accuracy: 0.9724
Epoch 43/70
4/4 [==============================] - 1s 194ms/step - loss: 0.0031 - accuracy: 0.9992 - val_loss: 0.4878 - val_accuracy: 0.9774
Epoch 44/70
4/4 [==============================] - 1s 200ms/step - loss: 5.5801e-04 - accuracy: 1.0000 - val_loss: 0.2931 - val_accuracy: 0.9849
Epoch 45/70
4/4 [==============================] - 1s 196ms/step - loss: 3.9625e-04 - accuracy: 1.0000 - val_loss: 0.2190 - val_accuracy: 0.9900
Epoch 46/70
4/4 [==============================] - 1s 197ms/step - loss: 4.2008e-04 - accuracy: 1.0000 - val_loss: 0.2155 - val_accuracy: 0.9916
Epoch 47/70
4/4 [==============================] - 1s 193ms/step - loss: 1.0929e-04 - accuracy: 1.0000 - val_loss: 0.2591 - val_accuracy: 0.99

In [14]:
y_test = model.predict_classes(x_test, verbose=1)
submission = pd.DataFrame({'vid_id': test['vid_id'].values, 'is_turkey': list(y_test.flatten())})

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
38/38 [==============================] - 1s 14ms/step


In [15]:
submission.to_csv("turkey_submission.csv", index=False)